### Halo check
Plot halos to see if halofinders work well

In [1]:
#import os
#base = os.path.abspath('/home/hoseung/Work/data/05427/')

#base = base + '/'

# basic parameters
wdir = '/home/hoseung/Work/data/05427/'
cluster_name = wdir.split('/')[-2]

frefine= 'refine_params.txt'
fnml = 'cosmo_200.nml'
nout_ini=186
nout_fi=187
nouts = range(nout_ini,nout_fi+1)

scale = 0.10
npix = 800

m_halo_min = 5e9

# data loading parameters
ptype=["star pos mass"]
refine_params = True
dmo=False
draw=True
draw_halos=True
draw_part = True
draw_hydro = False
if draw_hydro:
    lmax=19

In [2]:
import load
import utils.sampling as smp
import utils.match as mtc
import draw
import pickle


for nout in nouts:
    snout = str(nout).zfill(3)

    if refine_params:
        # instead of calculating zoomin region, just load it from the refine_params.txt file.
        # region = s.part.search_zoomin(scale=0.5, load=True)

        rr = load.info.RefineParam()
        rr.loadRegion(wdir + frefine)
      
        nn = load.info.Nml(fname=wdir + fnml)
        
        aexp = nn.aout[nout-1]
        i_aexp = mtc.closest(aexp, rr.aexp)
        
        x_refine = rr.x_refine[i_aexp]
        y_refine = rr.y_refine[i_aexp]
        z_refine = rr.z_refine[i_aexp]
        r_refine = rr.r_refine[i_aexp] * 0.5
        
        region = smp.set_region(xc = x_refine, yc = y_refine, zc = z_refine,
                                radius = r_refine * scale)
    else:   
        region = smp.set_region(xc=0.5, yc=0.5, zc=0.5, radius=0.1)   
        
    print(region)
    s = load.sim.Sim(nout, wdir, dmo=dmo, ranges=region["ranges"], setup=True)
    imgs = draw.img_obj.MapSet(info=s.info, region=region)
    imgp = draw.img_obj.MapImg(info=s.info, proj='z', npix=npix, ptype=ptype)
    imgp.set_region(region)

    #%%
    if draw_part:
        s.add_part(ptype)
        s.part.load()
        part = getattr(s.part, s.part.pt[0])
    
        x = part['x']
        y = part['y']
        z = part['z']
        m = part['m'] * s.info.msun # part must be normalized already!
    
        #imgp.set_data(draw.pp.den2d(x, y, z, m, npix, s.info, cic=True, norm_integer=True))
        imgp.set_data(draw.pp.den2d(x, y, z, m, npix, region, cic=True, norm_integer=True))
        imgs.ptden2d = imgp
#    imgp.show_data()

    #%%
    if draw_hydro:
        s.add_hydro()
        s.hydro.amr2cell(lmax=lmax)
#        field = draw.pp.pp_cell(s.hydro.cell, npix, s.info, verbose=True)
        ptype = 'gas_den'
        imgh = draw.img_obj.MapImg(info=s.info, proj='z', npix=npix, ptype=ptype)
        imgh.set_data(draw.pp.pp_cell(s.hydro.cell, npix, s.info, verbose=True))
        imgh.set_region(region)
    #    imgh.show_data()
        imgs.hydro = imgh
    
    #%%
    fdump = wdir + snout + 'map.pickle'
    with open(fdump, 'wb') as f:
        pickle.dump(imgs, f)
    if draw:
        if draw_part:
            imgs.ptden2d.plot_2d_den(save=wdir + cluster_name + snout +'star.png',
                                     vmin=1e13, vmax=1e20, dpi=400, show=False)
        if draw_hydro:    
            imgs.hydro.plot_2d_den(save=wdir + cluster_name +snout + 'hydro.png',
                                   vmax=15, vmin=10, show=False, dpi=400)

{'yr': [0.310907055262, 0.314907055262], 'xc': 0.506942387092, 'radius': 0.002, 'zr': [0.301108906833, 0.305108906833], 'zc': 0.303108906833, 'centers': [0.506942387092, 0.312907055262, 0.303108906833], 'ranges': [[0.504942387092, 0.508942387092], [0.310907055262, 0.314907055262], [0.301108906833, 0.305108906833]], 'xr': [0.504942387092, 0.508942387092], 'yc': 0.312907055262}
Ranges = [[0.504942387092, 0.508942387092], [0.310907055262, 0.314907055262], [0.301108906833, 0.305108906833]]

No AMR instance,
Loading one...
An AMR instance is created

Updating info.cpus
An AMR instance is created

Updating info.cpus
 Simulation set up.
Updating data
Types of particles you want to load are:  ['star pos mass']
No AMR instance,
Loading one...
An AMR instance is created

Updating info.cpus
No info._set_cpus attribute??
A particle instance is created

Use part.load() to load particle
Loading particle... 
 ranges: [[0.504942387092, 0.508942387092], [0.310907055262, 0.314907055262], [0.301108906833

In [3]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax1 = fig.add_subplot(111)
snout = str(nout).zfill(3)

fin = wdir + snout + 'map.pickle'
with open(fin, 'rb') as f:
    img = pickle.load(f)

ptimg = img.ptden2d
fout = wdir + snout + "dmmap_" + ptimg.proj + ".png"
img.ptden2d.plot_2d_den(save=False, show=False, vmin=1e13, vmax=1e20, dpi=400, axes=ax1)

import tree
import numpy as  np

#s = load.sim.Sim(nout, base_dir)
info = load.info.Info(nout=nout, base=wdir, load=True)
hall = tree.halomodule.Halo(nout=nout, base=wdir, halofinder="HM", info=info, load=True)

i_center = np.where(hall.data['np'] == max(hall.data['np']))
h = tree.halomodule.Halo()
h.derive_from(hall, [i_center]) 

#region = smp.set_region(xc=h.data.x, yc=h.data.y, zc=h.data.z, radius = h.data.rvir * 2)      

#%%
from draw import pp

ind = np.where(hall.data.mvir > 5e10)
h_sub = tree.halomodule.Halo()
h_sub.derive_from(hall, ind) 
#x = hall.data.x#[ind]
#y = hall.data.y#[ind]
#r = hall.data.rvir#[ind]
#pp.circle_scatter(ax1, x*npix, y*npix, r*30, facecolor='none', edgecolor='b', label='555')

#ax1.set_xlim(right=npix).
#ax1.set_ylim(top=npix)
pp.pp_halo(h_sub, npix, region=img.ptden2d.region, axes=ax1, rscale=3, name=True)

plt.show()

Are these parameters right?
Are these parameters right?


##### Load halofinder result

##### get position and virial radius 

##### load particles data (star or DM) and draw density map
##### plot halos on top of particle density map

Are these parameters right?
Are these parameters right?
